<a href="https://colab.research.google.com/github/SrinivasSelvaraj/MACHINE-LEARNING-IN-PYTHON-PROBLEMS-/blob/main/machine%20learning%20analysis%20on%20startup%20data%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1 LOAD THE STARTUPS DATA SETS

In [ ]:
import pandas as pd
startups_df = pd.read_csv("/content/50_Startups.csv")
startups_df.info()
startups_df.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


(50, 5)

2. Display the initial 10 columns for the first 5 records

In [ ]:
 startups_df.iloc[0:5, 0:10]

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


3. Use only the following features to predict The profit.

In [ ]:
print(startups_df.columns)

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit'], dtype='object')


In [ ]:
X_features = ['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit']
print(X_features)

['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit']


4. Some of these features are categorical. So encode them using one-hot encoding. A categorical feature with k values is encoded using (k-1) binary/dummy features.

In [ ]:
categorical_features = ['State']
startups_encoded_df = pd.get_dummies(startups_df[X_features], columns=categorical_features, drop_first = True)
X_features = startups_encoded_df.columns
print(X_features)

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'Profit',
       'State_Florida', 'State_New York'],
      dtype='object')


5. Add the constant column to estimate Beta 0 using statsmodel library. Split the dataset into Train and Validation Sets

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
X = sm.add_constant(startups_encoded_df)
Y = startups_df['Profit']
print(X)

train_X, test_X, train_y, test_y = train_test_split(X,Y,train_size = 0.8, random_state = 50)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


    const  R&D Spend  Administration  ...     Profit  State_Florida  State_New York
0     1.0  165349.20       136897.80  ...  192261.83              0               1
1     1.0  162597.70       151377.59  ...  191792.06              0               0
2     1.0  153441.51       101145.55  ...  191050.39              1               0
3     1.0  144372.41       118671.85  ...  182901.99              0               1
4     1.0  142107.34        91391.77  ...  166187.94              1               0
5     1.0  131876.90        99814.71  ...  156991.12              0               1
6     1.0  134615.46       147198.87  ...  156122.51              0               0
7     1.0  130298.13       145530.06  ...  155752.60              1               0
8     1.0  120542.52       148718.95  ...  152211.77              0               1
9     1.0  123334.88       108679.17  ...  149759.96              0               0
10    1.0  101913.08       110594.11  ...  146121.95              1         

6. Build the model on training set

In [ ]:
startups_mlr_model1 = sm.OLS(train_y,train_X).fit()
startups_mlr_model1.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Results: Ordinary least squares
===========================================================================
Model:                  OLS                Adj. R-squared:       1.000     
Dependent Variable:     Profit             AIC:                  -1720.6705
Date:                   2021-11-06 17:57   BIC:                  -1708.8483
No. Observations:       40                 Log-Likelihood:       867.34    
Df Model:               6                  F-statistic:          1.102e+30 
Df Residuals:           33                 Prob (F-statistic):   0.00      
R-squared:              1.000              Scale:                1.0402e-20
---------------------------------------------------------------------------
                 Coef.  Std.Err.          t           P>|t|   [0.025 0.975]
---------------------------------------------------------------------------
const           -0.0000   0.0000              -0.0584 0.9538 -0.0000 0.0000
R&D Spend       -0.0000   0.0000              -0.0671 0.9469 -0.0000 0.0000
Administration  -0.0000   0.0000              -0.2512 0.8032 -0.0000 0.0000
Marketing Spend  0.0000   0.0000               0.4100 0.6844 -0.0000 0.0000
Profit           1.0000   0.0000 545676362713275.0625 0.0000  1.0000 1.0000
State_Florida    0.0000   0.0000               0.0436 0.9655 -0.0000 0.0000
State_New York  -0.0000   0.0000              -0.0852 0.9326 -0.0000 0.0000
---------------------------------------------------------------------------
Omnibus:                 1.000           Durbin-Watson:             0.465  
Prob(Omnibus):           0.607           Jarque-Bera (JB):          0.860  
Skew:                    -0.344          Prob(JB):                  0.650  
Kurtosis:                2.793           Condition No.:             2307305
===========================================================================
* The condition number is large (2e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

7. Validate this model on the test data. Find the regression parameters, r2_score and RMSE of this model1

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

pred_y = startups_mlr_model1.predict(test_X)
print("The r2_score of the model is", np.abs(r2_score(test_y, pred_y)))
print("The RMSE of the trained model is", np.sqrt(mean_squared_error(test_y, pred_y)))
print("The model parameters are")
print(startups_mlr_model1.params)

The r2_score of the model is 1.0
The RMSE of the trained model is 9.586609138641857e-11
The model parameters are
const             -7.275958e-12
R&D Spend         -1.110223e-16
Administration    -1.665335e-16
Marketing Spend    8.326673e-17
Profit             1.000000e+00
State_Florida      1.818989e-12
State_New York    -3.637979e-12
dtype: float64
